# Spelling playground

Source: [Peter Norvig](http://norvig.com/spell-correct.html)

`big.txt` is "a concatenation of public domain book excerpts from Project Gutenberg and lists of most frequent words from Wiktionary and the British National Corpus."

In [1]:
import re
from collections import Counter

In [2]:
def words(text): return re.findall(r'\w+', text.lower())

In [66]:
# Load big text file with many English words
# Download from http://norvig.com/big.txt
WORDS = Counter(words(open('../big.txt').read()))
#WORDS = Counter(words(open('../data/test.txt').read()))

In [67]:
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Example

In [68]:
correction('spling')

'spring'

In [69]:
candidates('spling')

{'sling', 'splint', 'spring', 'spying'}

In [70]:
for x in candidates('spling'):
    print(x, P(x))

splint 1.5238641609559111e-05
spring 6.274734780406692e-05
sling 1.7927813658304835e-06
spying 8.963906829152417e-07


In [71]:
known(edits1('spling'))

{'sling', 'splint', 'spring', 'spying'}

# Example

In [73]:
correction('korrcted')

'corrected'

In [74]:
candidates('korrcted')

{'corrected'}

In [76]:
P('corrected')

1.2549469560813385e-05

In [77]:
P('the')

0.07154004401278254

In [78]:
WORDS.most_common(5)

[('the', 79809), ('of', 40024), ('and', 38312), ('to', 28765), ('in', 22023)]

In [65]:
len(WORDS)

2736

# Medical Examples

In [16]:
TRAIN = Counter(words(open('../data/train.txt').read()))
TEST = Counter(words(open('../data/test.txt').read()))

len(TRAIN), len(TEST)

(130046, 2736)

In [17]:
TRAIN.most_common(20)

[('of', 149453),
 ('and', 86792),
 ('the', 76629),
 ('au', 60815),
 ('fau', 60445),
 ('ad', 58283),
 ('in', 50447),
 ('university', 35441),
 ('cancer', 35338),
 ('a', 32975),
 ('to', 29119),
 ('2020', 25324),
 ('department', 25241),
 ('is', 21990),
 ('for', 21419),
 ('with', 20943),
 ('10', 20320),
 ('medical', 20084),
 ('medicine', 17516),
 ('hospital', 16090)]

In [18]:
TEST.most_common(20)

[('the', 310),
 ('of', 305),
 ('and', 221),
 ('in', 218),
 ('a', 202),
 ('j', 174),
 ('to', 172),
 ('m', 163),
 ('s', 139),
 ('t', 138),
 ('cancer', 136),
 ('tumor', 124),
 ('crossref', 115),
 ('cell', 103),
 ('resistance', 101),
 ('cells', 100),
 ('c', 84),
 ('1', 80),
 ('with', 78),
 ('e', 74)]

In [19]:
WORDS.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [20]:
def get_most_common_words(K=20):
    return([word for word, count in WORDS.most_common(K)])    

In [21]:
most_common = get_most_common_words(50)

In [22]:
[word for word, count in TEST.most_common(20) if word not in most_common]

['j',
 'm',
 't',
 'cancer',
 'tumor',
 'crossref',
 'cell',
 'resistance',
 'cells',
 'c',
 '1',
 'e']

In [23]:
(TRAIN - WORDS).most_common(20)

[('of', 109429),
 ('au', 60797),
 ('fau', 60445),
 ('ad', 58282),
 ('and', 48480),
 ('university', 35410),
 ('cancer', 35184),
 ('in', 28424),
 ('2020', 25324),
 ('department', 25207),
 ('10', 20226),
 ('medical', 20062),
 ('medicine', 17490),
 ('hospital', 16044),
 ('cell', 15898),
 ('0000', 15749),
 ('phst', 15529),
 ('china', 15465),
 ('doi', 14947),
 ('for', 14478)]

In [24]:
(TEST - WORDS).most_common(20)

[('tumor', 123),
 ('crossref', 115),
 ('cell', 91),
 ('j', 85),
 ('immunotherapy', 62),
 ('al', 57),
 ('resistance', 54),
 ('immune', 48),
 ('pubmed', 47),
 ('et', 39),
 ('pd', 37),
 ('2018', 34),
 ('therapy', 34),
 ('mechanisms', 33),
 ('antigen', 33),
 ('melanoma', 27),
 ('signaling', 25),
 ('2016', 23),
 ('tumors', 21),
 ('pathway', 21)]

In [25]:
(TRAIN - TEST).most_common(10)

[('of', 149148),
 ('and', 86571),
 ('the', 76319),
 ('au', 60815),
 ('fau', 60445),
 ('ad', 58283),
 ('in', 50229),
 ('university', 35440),
 ('cancer', 35202),
 ('a', 32773)]

In [26]:
(TRAIN | TEST).most_common(10)

[('of', 149453),
 ('and', 86792),
 ('the', 76629),
 ('au', 60815),
 ('fau', 60445),
 ('ad', 58283),
 ('in', 50447),
 ('university', 35441),
 ('cancer', 35338),
 ('a', 32975)]

# Tests

In [27]:
# some numbers changed... easter egg on bottom of big.txt ?
def unit_tests():
    assert correction('speling') == 'spelling'              # insert
    assert correction('korrectud') == 'corrected'           # replace 2
    assert correction('bycycle') == 'bicycle'               # replace
    assert correction('inconvient') == 'inconvenient'       # insert 2
    assert correction('arrainged') == 'arranged'            # delete
    assert correction('peotry') =='poetry'                  # transpose
    assert correction('peotryy') =='poetry'                 # transpose + delete
    assert correction('word') == 'word'                     # known
    assert correction('quintessential') == 'quintessential' # unknown
    assert words('This is a TEST.') == ['this', 'is', 'a', 'test']
    assert Counter(words('This is a test. 123; A TEST this is.')) == (
           Counter({'123': 1, 'a': 2, 'is': 2, 'test': 2, 'this': 2}))
    assert len(WORDS) == 32198
    assert sum(WORDS.values()) == 1115585
    assert WORDS.most_common(10) == [
     ('the', 79809),
     ('of', 40024),
     ('and', 38312),
     ('to', 28765),
     ('in', 22023),
     ('a', 21124),
     ('that', 12512),
     ('he', 12401),
     ('was', 11410),
     ('it', 10681)]
    assert WORDS['the'] == 79809
    assert P('quintessential') == 0
    assert 0.07 < P('the') < 0.08
    return 'unit_tests pass'

def spelltest(tests, verbose=False):
    "Run correction(wrong) on all (right, wrong) pairs; report results."
    import time
    start = time.clock()
    good, unknown = 0, 0
    n = len(tests)
    for right, wrong in tests:
        w = correction(wrong)
        good += (w == right)
        if w != right:
            unknown += (right not in WORDS)
            if verbose:
                print('correction({}) => {} ({}); expected {} ({})'
                      .format(wrong, w, WORDS[w], right, WORDS[right]))
    dt = time.clock() - start
    print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
          .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
    "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
    return [(right, wrong)
            for (right, wrongs) in (line.split(':') for line in lines)
            for wrong in wrongs.split()]

print(unit_tests())

unit_tests pass
